In [1]:
# ---- RUN THE FOLLOWING SCRIPT TO LAUNCH DASHBOARD ----

# ---- IMPORT LIBRARIES ----
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import holoviews as hv
import time
from tornado.ioloop import IOLoop

# Initialise extensions
pn.extension("tabulator")
hv.extension("bokeh")

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

# ---- DATA PREPROCESSING ----

# Load data frame
data = pd.read_csv("encoded_consolidated_movies.csv")

# Initialise list with features to be visualised
features_to_visualise = ['movieId', 'userId', 'rating', 'timestamp_x']

# Filter data frame
df = data[features_to_visualise]

# Ensure that rating values are floats to one decimal place
def value_to_float(v):
    return round(float(v),1)

df["rating"] = df["rating"].apply(value_to_float)

# Convert timestamp into datetime object
df["timestamp_x"] = pd.to_datetime(df["timestamp_x"], unit="s")

# Convert timestamp into date format (YYYY-MM)
df["timestamp_x"] = df["timestamp_x"].dt.strftime('%Y-%m')

# Sort data frame by date in ascending order
df = df.sort_values(by="timestamp_x", ascending=True)

# Show data set preview
print("\nFILTERED DATA FRAME:")
print("--------------------")
display(df.head())
print(f"Length of filtered data frame: {df.shape[0]}")
print()

# Save data frame in CSV format
df.to_csv("dashboard.csv", index=False)

# ---- SET UP CONTROL PANEL WIDGETS ----

# Initialise variables with min and max user ratings
rating_max = df["rating"].max()
rating_min = df["rating"].min()

# Initialise default control values
DEFAULT_SLIDER_POSITION = 5.0
DEFAULT_RADIO_BUTTON = "movieId"

# Create pane to display update alerts
update_message = pn.pane.HTML("")

# Create slider for user ratings
ratings_slider = pn.widgets.FloatSlider(name="User Ratings", start=rating_min, end=rating_max, step=0.5, value=rating_max)

# Create radio buttons for MovieID and UserID
yaxis_id_radio_btn = pn.widgets.RadioButtonGroup(
    name="Y-axis on line plot",
    options=["movieId", "userId"],
    button_type="success"
)

# Create refresh button to force immediate dashboard update upon clicking
refresh_btn = pn.widgets.Button(
    name="Refresh Button",
    button_type="primary"
)

# Create stop button to force immediate termination of dashboard refresh
stop_btn = pn.widgets.Button(
    name="Stop Button",
    button_type="danger"
)

# Initialise global function to track whether dashboard is being updated
dashboard_update = False

# ---- LOAD PREPROCESSED FILE ----

# Create dictionary to store visualisation titles
plot_titles = {
    "lp_title": {"movieId": ["Number of Movies Watched Over Time", "Number of Movies"],
                 "userId": ["Number of Active Users Over Time", "Number of Users"]},
    "bp_title": {"movieId": "Most Popular Movies", "userId": "Most Active Users"},
    "hg_title": {"movieId": ["Distribution of Movies", "Frequency of Movies"],
                 "userId": ["Distribution of Users", "Frequency of Users"]},
    "table_title": {"movieId": "Most Popular Movies", "userId": "Most Active Users"}
}

# Load preprocessed data frame
df = pd.read_csv("dashboard.csv")
df.head()

# ---- REACTIVE FUNCTIONS TO UPDATE DASHBOARD ----

def dynamic_bar_plot(user_rating, yaxis_id_selector, bp_title):
    """
    Dynamically updates bar plot based on selected user rating and radio button identifier.

    Parameters:
        user_rating (float): User rating corresponding to slider control selection.
        yaxis_id_selector (str): Identifier (e.g., movieID, userID) corresponding to radio button selection.
        title (str): Descriptive title for bar plot.

    Returns:
        bar_plot: Updated bar plot.
    
    """
    # Filter data frame according to rating selected by slider control within specified tolerance (closest match)
    filtered_df = df[np.isclose(df['rating'], user_rating, atol=0.01)]

    if yaxis_id_selector == "movieId":
        movies_id_given_rating = filtered_df["movieId"].value_counts(ascending=False).head(10).index.tolist()
        count_unique_users = filtered_df[filtered_df["movieId"].isin(movies_id_given_rating)].groupby("movieId")["userId"].nunique().sort_values(ascending=False)
        _df = count_unique_users.reset_index().rename(columns={"movieId": "Movie ID", "userId": "Number of individual users"})
        # Convert "Movie ID" to categorical values (string objects)
        _df["Movie ID"] = _df["Movie ID"].astype("str")
        #display(_df)
        #display(f"Movie ID Data type: {type(_df["Movie ID"].iloc[0])}")
        
        bar_plot = _df.hvplot.bar(
        x="Movie ID",
        y="Number of individual users",
        title=bp_title,
        line_width=1.5,
        width=800,
        height=500,
        bar_width=0.85, # Determine width of each bar
        fontsize={"title": "16px", "xlabel": "12px", "ylabel": "12px", "ticks": "0px"},
        legend=False
    ).opts(xrotation=45)

    elif yaxis_id_selector == "userId":
        user_id_given_rating = filtered_df["userId"].value_counts(ascending=False).head(10).index.tolist()
        count_unique_users = filtered_df[filtered_df["userId"].isin(user_id_given_rating)].groupby("userId")["movieId"].nunique().sort_values(ascending=False)
        #_df = count_unique_users.reset_index()
        _df = count_unique_users.reset_index().rename(columns={"userId": "User ID", "movieId": "Number of movies watched per user"})
        
        # Convert "User ID" to categorical values
        _df["User ID"] = _df["User ID"].astype("str")
        #display(_df)
        #display(f"User ID Data type: {type(_df["User ID"].iloc[0])}")
        
        bar_plot = _df.hvplot.bar(
        x="User ID",
        y="Number of movies watched per user",
        title=bp_title,
        line_width=1.5,
        width=800,
        height=500,
        bar_width=0.85, # Determine width of each bar
        fontsize={"title": "16px", "xlabel": "12px", "ylabel": "12px", "ticks": "0px"},
        legend=False
    ).opts(xrotation=45)

    return bar_plot


def dynamic_table_view(user_rating, yaxis_id_selector, table_title):
    """
    Dynamically updates table view based on selected user rating and radio button identifier.

    Parameters:
        user_rating (float): User rating corresponding to slider control selection.
        yaxis_id_selector (str): Identifier (e.g., movieID, userID) corresponding to radio button selection.
        title (str): Descriptive title for bar plot.

    Returns:
        table_view: Updated table view.
    
    """

    encoding_results = {'movies.csv': 'ISO-8859-1', 'rating.csv': 'ascii', 'tags.csv': 'Windows-1252'}

    # Filter data frame according to rating selected by slider control within specified tolerance (closest match)
    filtered_df = df[np.isclose(df['rating'], user_rating, atol=0.01)]

    if yaxis_id_selector == "movieId":
        
        # Determine movie IDs corresponding to top 10 movies data frame
        movies_id_given_rating = filtered_df["movieId"].value_counts(ascending=False).head(10).index.tolist()
        
        # Count number of individual users who have watched top 10 movies for specified rating
        count_unique_users = filtered_df[filtered_df["movieId"].isin(movies_id_given_rating)].groupby("movieId")["userId"].nunique().sort_values(ascending=False)
        # Convert data series to data frame and rename columns
        count_unique_users_df = count_unique_users.reset_index().rename(columns={"movieId": "Movie ID", "userId": "Unique Users Count"})

        # Load data frame containing movie titles
        movie_titles_df = pd.read_csv("movies.csv", encoding=encoding_results["movies.csv"])

        # Drop redundant feature
        if "genres" in movie_titles_df.columns:
            movie_titles_df.drop(columns=["genres"], inplace=True)
        
        # Rename data frame columns
        movie_titles_df = movie_titles_df.rename(columns={"movieId": "Movie ID", "title": "Title"})
        
        # Filter data frame to show only movie titles corresponding to movie IDs for top 10 movies
        filtered_movie_titles_df = movie_titles_df[movie_titles_df["Movie ID"].isin(movies_id_given_rating)]
        
        # Merge data frames using inner join on key Movie ID and sort in descending order by Unique Users Count
        merged_df = filtered_movie_titles_df.merge(count_unique_users_df, on="Movie ID", how="inner").sort_values(by="Unique Users Count", \
                                                                                                                  ascending=False)

        # Convert Movie ID to string object
        merged_df["Movie ID"] = merged_df["Movie ID"].astype(str)
     
        # Visualise merged data frame as table
        table_view = merged_df.pipe(
            pn.widgets.Tabulator,
            pagination="remote",
            page_size=10,
            sizing_mode="stretch_width",
            show_index=False
        )

    elif yaxis_id_selector == "userId":
        # Determine user IDs corresponding to top 10 most active users
        users_id_given_rating = filtered_df["userId"].value_counts(ascending=False).head(10).index.tolist()
        
        # Count number of individual movies for specified rating watched by most active users
        count_unique_users = filtered_df[filtered_df["userId"].isin(users_id_given_rating)].groupby("userId")["movieId"].nunique().sort_values(ascending=False)
        
        # Convert data series to data frame and rename columns
        count_unique_users_df = count_unique_users.reset_index().rename(columns={"userId": "User ID", "movieId": "Number of movies watched per user"})

        # Convert User ID to string object
        count_unique_users_df["User ID"] = count_unique_users_df["User ID"].astype(str)
        
        # Visualise merged data frame as table
        table_view = count_unique_users_df.pipe(
            pn.widgets.Tabulator,
            pagination="remote",
            page_size=10,
            sizing_mode="stretch_width",
            show_index=False
        )
        
    return table_view


def dynamic_line_plot(user_rating, yaxis_id_selector, lp_title):
    """
    Dynamically updates line plot based on selected user rating and radio button identifier.

    Parameters:
        user_rating (float): User rating corresponding to slider control selection.
        yaxis_id_selector (str): Identifier (e.g., movieID, userID) corresponding to radio button selection.
        title (str): Descriptive title for line plot.

    Returns:
        line_plot: Updated line plot.
        
    """
    # Filter data frame according to rating selected by slider control within specified tolerance (closest match)
    filtered_df = df[np.isclose(df['rating'], user_rating, atol=0.01)]

    # Count occurrences of selected option per timestamp
    grouped_df = filtered_df.groupby("timestamp_x")[yaxis_id_selector].nunique().reset_index()
    
    # Visualise grouped data using line plot
    line_plot = grouped_df.hvplot.line(
        x="timestamp_x", 
        y=yaxis_id_selector, 
        title=lp_title[0],
        xlabel="Timestamp",
        ylabel=lp_title[1],
        width=800, 
        height=500,
        line_width=2,
        fontsize={"title": "16px", "xlabel": "12px", "ylabel": "12px", "ticks": "0px"}
    )
    return line_plot


def dynamic_histogram(user_rating, yaxis_id_selector, hg_title):
    """
    Dynamically updates histogram based on selected user rating and radio button identifier.

    Parameters:
        user_rating (float): User rating corresponding to slider control selection.
        yaxis_id_selector (str): Identifier (e.g., movieID, userID) corresponding to radio button selection.
        title (str): Descriptive title for histogram.

    Returns:
        histogram: Updated histogram.
        
    """
    # Filter data frame according to rating selected by slider control within specified tolerance (closest match)
    filtered_df = df[np.isclose(df['rating'], user_rating, atol=0.01)]

    if yaxis_id_selector == "movieId":
        # Filter data frame to remove extreme values (outliers)
        filtered_df = filtered_df[filtered_df[yaxis_id_selector] <= 350]
    elif yaxis_id_selector == "userId":
        return None

    # Visualise filtered data using histogram
    histogram = filtered_df.hvplot.hist(
        y=yaxis_id_selector,
        bins=8,
        title=hg_title[0],
        xlabel="Number of Bins",
        ylabel=hg_title[1],
        width=800, 
        height=500,
        fontsize={"title": "16px", "xlabel": "12px", "ylabel": "12px", "ticks": "0px"}
    )
    
    return histogram


# Add decorator to call function each time user interacts with control
@pn.depends(ratings_slider.param.value, yaxis_id_radio_btn.param.value, refresh_btn.param.clicks, stop_btn.param.clicks) 
def refresh_dashboard(user_rating, yaxis_id_selector, refresh_clicks, stop_event):
    """
    Dynamically updates dashboard user interface each time user clicks on refresh button.

    Parameters:
        user_rating (float): User rating corresponding to slider control selection.
        yaxis_id_selector (str): Identifier (e.g., movieID, userID) corresponding to radio button selection.
        refresh_clicks (int): Number of times user clicks on refresh button.

    Returns:
        grid_layout: Updated visualisations in grid.

    """
    global dashboard_update
    
    # Wait for 7 seconds before updating dashboard, unless user clicks Refresh button
    if refresh_clicks == 0:
        time.sleep(7)

    # Reset flag when dashboard refresh is activated
    dashboard_update = True

    # Before dashboard updates, disable control panel widgets
    ratings_slider.disabled = True
    yaxis_id_radio_btn.disabled = True
    #refresh_btn.disabled = True
    stop_btn.disabled = False

    # Show notification above control panel while dashboard is updating
    update_message.object = ("<div style='color:green;font-size:14px;'><b>The dashboard is updating. Please wait ...</b></div>")

    # Check if dashboard update has not been terminated by user clicking Stop button
    if not dashboard_update:
        print("Dashboard update has been halted.")
        return None

    # Re-calculate each plot using filtered data
    bar_plot = dynamic_bar_plot(user_rating, yaxis_id_selector, plot_titles["bp_title"][yaxis_id_selector])
    table_view = dynamic_table_view(user_rating, yaxis_id_selector, plot_titles["table_title"][yaxis_id_selector])
    line_plot = dynamic_line_plot(user_rating, yaxis_id_selector, plot_titles["lp_title"][yaxis_id_selector])
    histogram = dynamic_histogram(user_rating, yaxis_id_selector, plot_titles["hg_title"][yaxis_id_selector])

    # Prepare the grid layout before displaying visualisations
    grid_layout = pn.GridSpec(sizing_mode="stretch_both", max_height=1500)
    grid_layout[0, 0] = bar_plot
    grid_layout[0, 1] = table_view
    grid_layout[1, 0] = line_plot
    grid_layout[1, 1] = histogram

    # Unfreeze control panel after short delay once dashboard update has completed
    def enable_widgets():
        if dashboard_update:
            update_message.object = "" # Clear message after dashboard has completed update
            ratings_slider.disabled = False
            yaxis_id_radio_btn.disabled = False
            #refresh_btn.disabled = False
            stop_btn.disabled = True
    IOLoop.current().call_later(1.5, enable_widgets)

    return grid_layout

# ---- FUNCTION TO STOP DASHBOARD UPDATE ----

def stop_dashboard_update(event):
    global dashboard_update
    # Set flag to terminate dashboard refresh
    dashboard_update = False
    # Reset slider control to default value
    ratings_slider.value = DEFAULT_SLIDER_POSITION
    yaxis_id_radio_btn.value = DEFAULT_RADIO_BUTTON
    # Unfreeze control panel
    ratings_slider.disabled = False
    yaxis_id_radio_btn.disabled = False
    #refresh_btn.disabled = False
    stop_btn.disabled = True

# ---- ATTACH BUTTONS TO FUNCTIONS ----

#refresh_btn.on_click(refresh_dashboard)
stop_btn.on_click(stop_dashboard_update)

# ---- SET UP "WATCHER" FOR REFRESH BUTTON CLICKS ----

#refresh_btn.param.watch(refresh_dashboard, 'clicks')

# ---- SET UP PANEL TEMPLATE ----

# Create titles, sub-titles, and explanatory text for dashboard
dashboard_title = "Movie Ratings Dashboard"
dashboard_subtitle = "# Movie Popularity by User Rating"
dashboard_text = "#### Select a user rating to find out which movies are the most watched as well as other key statistics."
user_guide = "Please move the slider control and toggle the buttons Movie ID and User ID to refresh the dashboard."

template = pn.template.FastListTemplate(
    title=dashboard_title,
    sidebar=[
        update_message, # Dashboard refresh message appears on top of sidebar above controls
        pn.pane.Markdown(dashboard_subtitle),
        pn.pane.Markdown(dashboard_text),
        pn.pane.Markdown(user_guide, styles={"color": "red"}),
        pn.pane.Markdown("## Control Panel"),
        ratings_slider, # Add slider control to allow user to select specific rating
        yaxis_id_radio_btn, # Add radio buttons to allow user to select MovieId or UserId
        #refresh_btn,
        stop_btn
    ],
    main=[pn.panel(refresh_dashboard)], # Call function to dynamically update dashboard
    accent_base_color="#88d8b0",
    header_background="#88d8b0"
)

# ---- lAUNCH DASHBOARD ----
#template.servable()
pn.serve(template, port=8080, allow_websocket_origin="https://mybinder.org/v2/gh/Darren-SBA23255-CCT-L8/dashboard/main")


FILTERED DATA FRAME:
--------------------


,movieId,userId,rating,timestamp_x
8949,25,24797,5.0,1997-10
13175,41,1835,5.0,1997-10
13176,41,1835,5.0,1997-10
151601,969,1835,4.0,1997-10
26585,150,1835,4.0,1997-10


Length of filtered data frame: 906172

Launching server at http://localhost:8888
